In [1]:
# 음식점(한식) API 추출(1) - contentid, 위도/경도
# cat3(소분류) 항목 - 한식(A05020100), 양식(A05020200), 일식(A05020300), 중식(A05020400), 카페(A05020900)
# cat3(소분류) 항목 - 일식(A05020300)

import requests  # http request
import json
import pandas as pd

# 생략없이 전체 행 보기
pd.set_option('display.max_rows', None)

# TourAPI 인증키 통하여 json 형식으로 호출 (시군구 구분)
# 강원도 시/군구 총 18개

# 시군구 코드
numlist=[]
for i in range(1,19):
    numlist.append(str(i))

    # 시군구 관광지 별 3곳씩 추출
listdata=[]
for num in numlist:
    url = "https://apis.data.go.kr/B551011/KorService1/areaBasedList1?serviceKey=v7dGxSp5LTs8baa4MfO8DLOIKZ4%2B9IU7T3MAa3mdB8h11V5TOPe%2FJLyEfE0zfm3OOeXQtJJt9Uhc5uFn3zAeLQ%3D%3D&pageNo=1&numOfRows=10&MobileApp=AppTest&MobileOS=ETC&arrange=A&contentTypeId=39&cat1=A05&cat2=A0502&cat3=A05020300&areaCode=32&sigunguCode="+num+"&_type=json"
    r = requests.get(url)
    data = r.json()
    
    # 양식 음식점이 있는 경우 vs 없는 경우 나눠서 출력
    if data['response']['body']['items']=="": # 음식점이 없을때
        
        continue
    else: # 음식점이 있을때
        item = data['response']['body']['items']['item']
    
        # 대괄호 제외하고 listdata에 넣기
        if item:
            for j in range(len(item)):
                listdata.append(item[j])
            
listdata

# 주소, contentid, 장소명 추출 후 key:value 형태로 변환
newdictdata = [] 

for i in range(len(listdata)):  # 리스트의 길이만큼 반복
    contentid = listdata[i]['contentid']
    title = listdata[i]['title']
    addr = listdata[i]['addr1'] + listdata[i]['addr2']
    mapx = listdata[i]['mapx']
    mapy = listdata[i]['mapy']
    
    # 딕셔너리 형태로 추가
    item_dict = {
        'contentid': contentid,
        '장소명': title,
        '주소': addr,
        '경도': mapx,
        '위도': mapy
    }
    # 리스트에 추가
    newdictdata.append(item_dict) 
    
newdictdata

# list -> json 문자열 변환
json_str = json.dumps(newdictdata,ensure_ascii=False)

json_str

# json 문자열 -> df 변환
dfdata=pd.read_json(json_str) 
dfdata.head(5)

# 아래 url의 contentid 작동을 위해 int -> string 
dfdata=dfdata.astype({'contentid':'string'})

# contentid 칼럼 추출
dfdataci=dfdata.iloc[:,0]

dfdataci

# 위도/경도 칼럼 추출
dfdataci_map=dfdata.iloc[:,3:]
dfdataci_map

dfdata



,contentid,장소명,주소,경도,위도
0,2685487,그린횟집,강원특별자치도 강릉시 사천면 진리해변길 123,128.874483,37.842022
1,2775513,대영유통,강원특별자치도 강릉시 주문진읍 해안로 1767,128.828827,37.892904
2,2685554,동산항물회,강원특별자치도 강릉시 경강로2267번길 22,128.908076,37.767203
3,2775525,메시56,강원특별자치도 강릉시 초당순두부길 56(초당동),128.914622,37.788340
4,2685581,산횟집,강원특별자치도 강릉시 강동면 율곡로 1165,129.031381,37.690623
5,2943818,스시심,강원특별자치도 강릉시 하슬라로206번길 9,128.876921,37.768111
6,2670838,신라이사부독도참치,강원특별자치도 강릉시 솔올로 67,128.878673,37.768136
7,2660867,얼라이브홈,강원특별자치도 강릉시 강릉대로203번길 4-1,128.893164,37.758704
8,2685615,오무라안,강원특별자치도 강릉시 성덕포남로188번길 27,128.910990,37.769845
9,2872436,이츠모라멘,강원특별자치도 강릉시 초당원길 34,128.916095,37.784871


In [2]:
# 음식점(일식) API 추출(2)  - 강원도 시군구  장소명, 주소, 개요 정보

# contentid를 받아서 json 형식으로 호출
# contentid를 conid라는 변수로 받음

listdata2=[]
for index, conid in dfdataci.items():
    url = "http://apis.data.go.kr/B551011/KorService1/detailCommon1?serviceKey=2rTA0AsQYrJXYZI8zaF0J2zdy6kUHExuEVXoYPtGJbcUWAkiSpziy2aw%2BVpKwoyE5DFegvPyKwzFTuKcxY16%2Fw%3D%3D&contentId="+conid+"&defaultYN=Y&addrinfoYN=Y&overviewYN=Y&MobileOS=ETC&MobileApp=AppTest&_type=json"
    r = requests.get(url)
    data2 = r.json()   
    item = data2['response']['body']['items']['item']
    
    # 대괄호 제외하고 listdata2에 넣기
    if item: 
        listdata2.append(item[0])

listdata2

# TourAPI에서 강원도 음식점 양식 별 장소명, 주소, 개요 정보 가져오기
newdictdata2 = [] 
for i in range(len(listdata2)):  
    title = listdata2[i]['title']
    addr = listdata2[i]['addr1'] + listdata2[i]['addr2']
    overview = listdata2[i]['overview']
    
    # 주소에서 시군구 분류 추출
    sigungu = ''
    if '강릉' in addr:
        sigungu = '강릉시'
    elif '고성' in addr:
        sigungu = '고성군'
    elif '동해' in addr:
        sigungu = '동해시'
    elif '삼척' in addr:
        sigungu = '삼척시'
    elif '속초' in addr:
        sigungu = '속초시'
    elif '양구' in addr:
        sigungu = '양구군'
    elif '양양' in addr:
        sigungu = '양양군'
    elif '영월' in addr:
        sigungu = '영월군'
    elif '원주' in addr:
        sigungu = '원주시'
    elif '인제' in addr:
        sigungu = '인제군'
    elif '정선' in addr:
        sigungu = '정선군' 
    elif '철원' in addr:
        sigungu = '철원군'
    elif '춘천' in addr:
        sigungu = '춘천시'
    elif '태백' in addr:
        sigungu = '태백시'
    elif '평창' in addr:
        sigungu = '평창군'
    elif '홍천' in addr:
        sigungu = '홍천군'
    elif '화천' in addr:
        sigungu = '화천군'
    elif '횡성' in addr:
        sigungu = '횡성군'
    
    # 딕셔너리 형태로 추가
    item_dict = {
        '축제명': title,
        '시군구 분류': sigungu,
        '소재지지번주소': addr,
        '설명': overview
    }
    
    # 리스트에 추가
    newdictdata2.append(item_dict) 
    
# 결과 확인 => list
# newdictdata2

# list -> json 문자열 변환
json_str2 = json.dumps(newdictdata2,ensure_ascii=False)

# json 문자열 -> df 변환
dfdata2=pd.read_json(json_str2) 

# 위도/경도 칼럼 추가
dfdata_final=pd.concat([dfdata2,dfdataci_map],axis=1)
dfdata_final

# DB 연동 사전 작업 - list로 변환
all_df2=[]
for i in range(len(dfdata_final)):
    all_df2.append(dfdata_final.loc[i].to_list())
    
all_df2

dfdata_final 

dfdata_final.to_csv('일식.csv', index=False, encoding='utf-8')


In [3]:
# DB 연동 - 강원도 시군구 별 관광지 정보 insert
import cx_Oracle as cx
con2=cx.connect("hr","hr","localhost:1521/xe") 
cur2=con2.cursor() 
sql2="insert into food(place, sigungu, address, overview, longtitude, altitude) values(:1,:2,:3,:4,:5,:6)" 

# insert 내용들
for place, sigungu, address, overview, longtitude, altitude in all_df2:
    cur2.execute(sql2,(place,sigungu,address,overview,longtitude, altitude))

# commit
con2.commit() 